In [6]:
#import all needed libraries
import requests
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# enter key
key = {'X-API-Key' : os.getenv('PROPUBLICA_KEY')}

# gather congressmen demographic data
r_senate = requests.get('https://api.propublica.org/congress/v1/115/senate/members.json', headers = key)
r_house = requests.get('https://api.propublica.org/congress/v1/115/house/members.json', headers = key)

In [7]:
temp_senate = r_senate.json()
temp_house = r_house.json()

senate_json = temp_senate['results'][0]['members']
house_json = temp_house['results'][0]['members']


senate_df = pd.DataFrame(senate_json)
house_df = pd.DataFrame(house_json)

needed_headers = ["id","first_name","last_name","party","state","phone","votes_with_party_pct"]

In [8]:
# create a clean dataframe with a markers for which chamber of Congress they are a member of 
house_df_clean = house_df[needed_headers]
house_df_clean["chamber"] = "H"

senate_df_clean = senate_df[needed_headers]
senate_df_clean["chamber"] = "S"

# append the two dataframes to create one clean data frame
congress_members = house_df_clean.append(senate_df_clean)
congress_members.head()

/anaconda/envs/data_analysis/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/envs/data_analysis/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,first_name,last_name,party,state,phone,votes_with_party_pct,chamber
0,A000055,Robert,Aderholt,R,AL,202-225-4876,98.17,H
1,A000367,Justin,Amash,R,MI,202-225-3831,84.09,H
2,A000369,Mark,Amodei,R,NV,202-225-6155,97.69,H
3,A000370,Alma,Adams,D,NC,202-225-1510,97.73,H
4,A000371,Pete,Aguilar,D,CA,202-225-3201,95.02,H


In [9]:
# get summary voting records to each bill in the house
r_house_votes_1 = requests.get('https://api.propublica.org/congress/v1/house/votes/2017/01.json', headers = key).json()
r_house_votes_2 = requests.get('https://api.propublica.org/congress/v1/house/votes/2017/02.json', headers = key).json()
r_house_votes_3 = requests.get('https://api.propublica.org/congress/v1/house/votes/2017/03.json', headers = key).json()
r_house_votes_4 = requests.get('https://api.propublica.org/congress/v1/house/votes/2017/04.json', headers = key).json()

In [10]:
# create one large dataframe for all of the House votes

# check the number of bills
roll_calls_jan = len(r_house_votes_1['results']['votes'])
roll_calls_feb = len(r_house_votes_2['results']['votes'])
roll_calls_mar = len(r_house_votes_3['results']['votes'])
roll_calls_apr = len(r_house_votes_4['results']['votes'])
roll_calls_total = roll_calls_jan + roll_calls_feb + roll_calls_mar + roll_calls_apr

all_house_votes = pd.DataFrame()
for rc in tqdm(range(1, roll_calls_total)):
    try:
        rc_votes_temp_house = requests.get("https://api.propublica.org/congress/v1/115/house/sessions/1/votes/" + str(rc) + ".json", headers = key).json()
        temp_df = pd.DataFrame(rc_votes_temp_house['results']['votes']['vote']['positions'])
        all_house_votes = all_house_votes.append(temp_df)
        all_house_votes['bill_id'] = rc_votes_temp_house['results']['votes']['vote']['bill']['bill_id']
    except KeyError as error:
        print error


  0%|          | 1/220 [00:03<12:02,  3.30s/it]

'bill'




  1%|          | 2/220 [00:04<09:54,  2.73s/it]

'results'



 13%|█▎        | 28/220 [01:38<13:21,  4.17s/it]

...


 91%|█████████▏| 201/220 [11:28<01:12,  3.82s/it]

'bill'



100%|█████████▉| 219/220 [12:25<00:03,  3.81s/it]

'bill'



100%|██████████| 220/220 [12:28<00:00,  3.36s/it]


In [11]:
# check the structure of the dataset
all_house_votes.head()

,bill_id,dw_nominate,member_id,vote_position
0,hr1304-115,,A000374,Present
1,hr1304-115,,A000370,Present
2,hr1304-115,,A000055,Present
3,hr1304-115,,A000371,Present
4,hr1304-115,,A000372,Present


In [12]:
# get summary voting records to each bill in the house
r_senate_votes_1 = requests.get('https://api.propublica.org/congress/v1/senate/votes/2017/01.json', headers = key).json()
r_senate_votes_2 = requests.get('https://api.propublica.org/congress/v1/senate/votes/2017/02.json', headers = key).json()
r_senate_votes_3 = requests.get('https://api.propublica.org/congress/v1/senate/votes/2017/03.json', headers = key).json()
r_senate_votes_4 = requests.get('https://api.propublica.org/congress/v1/senate/votes/2017/04.json', headers = key).json()

In [13]:
# create one large dataframe for all of the Senate votes
senate_roll_calls_jan = len(r_senate_votes_1['results']['votes'])
senate_roll_calls_feb = len(r_senate_votes_2['results']['votes'])
senate_roll_calls_mar = len(r_senate_votes_3['results']['votes'])
senate_roll_calls_apr = len(r_senate_votes_4['results']['votes'])
senate_roll_calls_total = senate_roll_calls_jan + senate_roll_calls_feb + senate_roll_calls_mar + senate_roll_calls_apr

all_senate_votes = pd.DataFrame()
for rc in tqdm(range(1,senate_roll_calls_total)):
    print rc
    try:
        rc_votes_temp_senate = requests.get("https://api.propublica.org/congress/v1/115/senate/sessions/1/votes/" + str(rc) + ".json", headers = key).json()
        temp_df = pd.DataFrame(rc_votes_temp_senate['results']['votes']['vote']['positions'])
        all_senate_votes = all_senate_votes.append(temp_df)
        all_senate_votes['bill_id'] = rc_votes_temp_senate['results']['votes']['vote']['bill']['bill_id']
    except KeyError as error:
        print error

  0%|          | 0/110 [00:00<?, ?it/s]

1


  1%|          | 1/110 [00:00<01:37,  1.11it/s]

2


  2%|▏         | 2/110 [00:01<01:40,  1.07it/s]

...

'bill'
109


 99%|█████████▉| 109/110 [01:54<00:01,  1.12s/it]

'bill'
110


100%|██████████| 110/110 [01:55<00:00,  1.07s/it]

'bill'


In [ ]:
# check the structure of the senate dataset
all_senate_votes.head()

In [ ]:
# # check the number of House roll call votes in dataset
# print "Number of House votes: " + str(len(all_house_votes))
# print "Number of House roll call votes with Bill ID: " + str(len(all_house_votes) / 435)
# print "Number of House Bill IDs: " + str(len(all_house_votes['bill_id']))
# print "Number of House Unique Bill IDs: " + str(len(all_house_votes['bill_id'].unique()))

# # check the number of Senate roll call votes in dataset
# print "Number of Senate votes: " + str(len(all_senate_votes))
# print "Number of Senate roll call votes with Bill ID: " + str(len(all_senate_votes) / 100)
# print "Number of Senate Bill IDs: " + str(len(all_house_votes['bill_id']))
# print "Number of Senate Unique Bill IDs: " + str(len(all_house_votes['bill_id'].unique()))

In [ ]:
# merge in membes with votes
all_votes = pd.concat([all_house_votes,all_senate_votes])
all_votes['counter'] = 1

count_by_congressman = all_votes.groupby(['member_id','vote_position']).count()
count_by_congressman_df = pd.DataFrame(count_by_congressman['counter'])
count_by_congressman_df = count_by_congressman_df.reset_index(level = ['member_id','vote_position'])
count_by_congressman_df = count_by_congressman_df[count_by_congressman_df['vote_position'] != "Present"]
count_by_congressman_df.head()

In [ ]:
full_df = congress_members.merge(count_by_congressman_df, how = "inner", left_on = "id", right_on = "member_id")

full_df['first_name'] = full_df['first_name'].str.encode('utf-8', errors='strict')
full_df['last_name'] = full_df['last_name'].str.encode('utf-8', errors='strict')

full_df = full_df.drop(['member_id'], axis = 1)
print full_df.dtypes

In [ ]:
# full_df.to_csv('/Users/rmenon/Projects/congress_app/lib/data/congress_vote_distribution.csv',index = False)
full_df.to_json('/Users/rmenon/Projects/congress_app/lib/data/congress_vote_distribution.json')